<a href="https://colab.research.google.com/github/KOTEAYOON/colab/blob/main/04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이미지 파일 로딩  

In [2]:
import torch
import imageio
img_arr=imageio.imread('/content/drive/MyDrive/pytorch_deep/file/data/data/p1ch4/image-dog/bobby.jpg')
img_arr.shape

(720, 1280, 3)

In [3]:
img=torch.from_numpy(img_arr)
out=img.permute(2,0,1) # 레이아웃 변경하기 (img에서 픽셀을 변경하면 out에있는 픽셀도 바뀜)

In [4]:
import os
batch_size=3
batch=torch.zeros(batch_size,3,256,256,dtype=torch.uint8)
data_dir='/content/drive/MyDrive/pytorch_deep/file/data/data/p1ch4/image-cats'
filenames=[name for name in os.listdir(data_dir) \
           if os.path.splitext(name)[-1]=='.png']
for i,filename in enumerate(filenames):
  img_arr=imageio.imread(os.path.join(data_dir,filename))
  img_t=torch.from_numpy(img_arr)
  img_t=img_t.permute(2,0,1)
  img_t=img_t[:3]# 첫 3개의 채널 유지
  batch[i]=img_t

# 데이터 정규화  

In [5]:
batch=batch.float()
batch/=255.0
n_channels=batch.shape[1]
for c in range(n_channels):
  mean=torch.mean(batch[:,c])
  std=torch.std(batch[:,c])
  batch[:,c]=(batch[:,c]-mean)/std

# 와인데이터 

In [6]:
# import imageio
# dir_path='/content/drive/MyDrive/pytorch_deep/file/data/data/p1ch4/volumetric-dicom/2-LUNG 3.0  B70f-04083'
# vol_arr=imageio.volread(dir_path,'DICOM')

In [7]:
# delimiter:각행을 구별하는 구분자
import csv
import numpy as np

wine_path='/content/drive/MyDrive/pytorch_deep/file/data/data/p1ch4/tabular-wine/winequality-white.csv'
wineq_numpy=np.loadtxt(wine_path,dtype=np.float32,delimiter=';',skiprows=1)

col_list=next(csv.reader(open(wine_path),delimiter=';'))
wineq_numpy,col_list

# numpy 배열을 pytorch 로 변환
wineq=torch.from_numpy(wineq_numpy)

In [8]:
data=wineq[:,:-1]
print(data, data.shape)
print('-'*50)
target=wineq[:,-1]
print(target,target.shape)

tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
        [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
        [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
        ...,
        [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
        [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
        [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]) torch.Size([4898, 11])
--------------------------------------------------
tensor([6., 6., 6.,  ..., 6., 7., 6.]) torch.Size([4898])


In [9]:
target=wineq[:,-1].long()
target


tensor([6, 6, 6,  ..., 6, 7, 6])

In [10]:
target_onehot=torch.zeros(target.shape[0],10)
target_onehot.scatter_(1,target.unsqueeze(1),1.0)
# scatter_(축, 새로 나타낼 인덱스, 새로 저장할 입력값)


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [11]:
print(target.unsqueeze(1).shape)
print(target.shape)

torch.Size([4898, 1])
torch.Size([4898])


In [12]:
data_mean=torch.mean(data,dim=0)
data_var=torch.var(data,dim=0)
data_normalized=(data-data_mean)/torch.sqrt(data_var)
data_normalized

tensor([[ 1.7208e-01, -8.1761e-02,  2.1326e-01,  ..., -1.2468e+00,
         -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7996e-02,  ...,  7.3995e-01,
          1.3422e-03, -8.2419e-01],
        [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
         -4.3677e-01, -3.3663e-01],
        ...,
        [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
         -2.6153e-01, -9.0545e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
         -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
         -1.4882e+00,  1.0448e+00]])

In [13]:
#임계값으로 찾기
bad_indexes=target<=3
bad_indexes,bad_indexes.shape,bad_indexes.dtype,bad_indexes.sum()
bad_data=data[bad_indexes]
bad_data.shape

torch.Size([20, 11])

In [14]:
bad_data = data[target <= 3]
mid_data = data[(target > 3) & (target < 7)] # <1>
good_data = data[target >= 7]

bad_mean = torch.mean(bad_data, dim=0)
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data, dim=0)

for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


In [15]:
total_sulfur_threshold=141.83
total_sulfur_data=data[:,6]
predicted_indexes=torch.lt(total_sulfur_data,total_sulfur_threshold)
predicted_indexes.shape,predicted_indexes.dtype,predicted_indexes.sum()
# (torch.lt는 2개의 텐서를 비교했을때 작으면 True, 크거나 같으면 False를 반환)

(torch.Size([4898]), torch.bool, tensor(2727))

In [16]:
# 실제로 좋은 와인의 인덱스
actual_indexes=target>5
actual_indexes.shape,actual_indexes.dtype,actual_indexes.sum()
# 임계값으로 예측한 수량보다 좋은 와인이 500개 존재한다

(torch.Size([4898]), torch.bool, tensor(3258))

In [17]:
n_matches=torch.sum(actual_indexes&predicted_indexes).item()
n_predicted=torch.sum(predicted_indexes).item()
n_actual=torch.sum(actual_indexes).item()
n_matches,n_matches/n_predicted,n_matches/n_actual
# item()은 요소를 가져오는 함수

(2018, 0.74000733406674, 0.6193984039287906)